In [5]:
import scipy.optimize
import numpy as np

# returns in excess of risk free rate
asset_returns = np.array([
    .005,    # cash. 0.5% return per year (assumed borrow costs)
    .05,     # stocks, 5% per year
    .02,     # long term bonds. 2% per year.
    .015,    # total bond market
])

# volatility in excess of risk free rate
asset_volatility = np.array([
    .0,     # cash, no volatility
    .16,    # stocks, 16% volatility
    .10,    # long term bonds, 10% volatility
    .03,    # total bond market, 3% volatility
])

# correlations
stock_ltt_corr = 0
stock_tbm_corr = 0
ltt_tbm_corr = .5
asset_correlations = np.array([
    [1,              0,              0,              0],    # correlations with cash all zero
    [0,              1,              stock_ltt_corr, stock_tbm_corr],
    [0,              stock_ltt_corr, 1,              ltt_tbm_corr],
    [0,              stock_tbm_corr, ltt_tbm_corr,   1],
])

# gamma = 1 maximizes log returns
# gamma = 3 probably more realistic for the average investor
gamma = 1.0


def mean_and_std(asset_weights):
    r = np.dot(asset_weights, asset_returns)
    std = asset_weights * asset_volatility
    std = std.reshape((1, len(std)))
    var = np.matmul(np.matmul(std, asset_correlations), std.transpose())
    return r, var[0][0]**.5


def utility(asset_weights):
    mean, std = mean_and_std(asset_weights)
    return mean - .5 * gamma * std**2

cash_bounds = (-2, 0.5)
asset_bounds = (0, np.inf)

solution = scipy.optimize.minimize(
    lambda x: -utility(x),
    x0=np.full_like(asset_returns, .5),
    bounds=[cash_bounds] + [asset_bounds] * (len(asset_returns)-1),
    constraints=[
        {'type': 'eq', 'fun': lambda x: x.sum() - 1},   # constrain sum(weights) == 1
    ],
    method='SLSQP',
    tol=1e-20
)

print(solution.message)
print("asset weights: ", solution.x)
print("mean: {:6.2%}, std: {:6.2%}".format(*mean_and_std(solution.x)))

Optimization terminated successfully.
asset weights:  [-2.          1.43427915  0.5139959   1.05172495]
mean:  8.78%, std: 24.07%
